<a href="https://colab.research.google.com/github/utkarshg1/Neural-Net-Datasets/blob/main/agents_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain langgraph langchain-tavily langchain-groq

In [2]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [3]:
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

# create custom tools
@tool
def calculate_emi(p: str, n: str, r: str)-> dict[str, float]:
  """
  This tool calculates emi, interest and amount for given
  p: principal in INR
  n: Number of years
  r: Rate of interst in %p.a.
  """
  p = float(p)
  n = float(n)
  r = float(r)
  n = n*12
  r = r/1200
  x = (1 + r)**n
  emi = p*r*x/(x - 1)
  amt = emi*n
  i = amt - p
  return {
      "emi": emi,
      "amount": amt,
      "interest": i
  }

# create agent
memory = MemorySaver()
model = ChatGroq(model="llama-3.3-70b-versatile")
search = TavilySearch()
tools = [search, calculate_emi]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [4]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Can you search for temperature in pune today?",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Can you search for temperature in pune today?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (zt67c0kfb)
 Call ID: zt67c0kfb
  Args:
    query: Pune temperature today
    search_depth: basic
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "Pune temperature today", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://timesofindia.indiatimes.com/weather/pune-weather-forecast-today/411002", "title": "Pune Weather Today, Pune Temperature Hourly and Weekly Forecast", "content": "Today's Weather in Pune: In Pune today, the weather is expected to be Cloudy with a maximum temperature of 28°C and a minimum of 21°C. Sunrise in Pune is", "score": 0.93161833, "raw_content": null}, {"url": "https://www.timeanddate.com/weather/india/pune", "t

In [5]:
def get_response(query: str, thread_id: str):
  config = {"configurable": {"thread_id": thread_id}}
  input_message = {
    "role": "user",
    "content": query,
  }
  for step in agent_executor.stream(
      {"messages": [input_message]}, config, stream_mode="values"
  ):
      step["messages"][-1].pretty_print()


In [6]:
get_response("Calculate emi for principal of 10_00_000 rate of interst of 8.1 % p.a. for 5 years", "abc123")

================================ Human Message =================================

Calculate emi for principal of 10_00_000 rate of interst of 8.1 % p.a. for 5 years
================================== Ai Message ==================================
Tool Calls:
  calculate_emi (1gng6bwpx)
 Call ID: 1gng6bwpx
  Args:
    n: 5
    p: 1000000
    r: 8.1
================================= Tool Message =================================
Name: calculate_emi

{"emi": 20324.28652922556, "amount": 1219457.1917535337, "interest": 219457.19175353367}
================================== Ai Message ==================================

The EMI for a principal amount of 10,00,000, an interest rate of 8.1% p.a., and a tenure of 5 years is approximately 20,324.29. The total amount to be paid is 12,19,457.19, and the total interest paid is 2,19,457.19.


In [7]:
thread_id = "abc123"
get_response("What do you think is sentiment of banking stocks in india", thread_id)

================================ Human Message =================================

What do you think is sentiment of banking stocks in india
================================== Ai Message ==================================
Tool Calls:
  tavily_search (hc1mdbf3z)
 Call ID: hc1mdbf3z
  Args:
    query: sentiment of banking stocks in India
    search_depth: advanced
    topic: finance
================================= Tool Message =================================
Name: tavily_search

{"query": "sentiment of banking stocks in India", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.bloomberg.com/news/articles/2025-07-21/india-market-buzz-reliance-sentiment-may-hinge-on-jio-moves", "title": "Investors Focus on Indian Bank Stocks After Earnings ...", "content": "*   [Software Updates](https://www.bloomberg.com/professional/support/customer-support/software-updates/?utm_source=bloomberg-menu&utm_medium=bcom) *   [About](https://www.bloomberg.com/compa

In [9]:
get_response("Can you tell me which sector will grow in upcoming decade in India?", thread_id)

================================ Human Message =================================

Can you tell me which sector will grow in upcoming decade in India?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (73rc2e2dx)
 Call ID: 73rc2e2dx
  Args:
    query: sectors to grow in India in the next decade
    search_depth: advanced
    topic: finance
================================= Tool Message =================================
Name: tavily_search

{"query": "sectors to grow in India in the next decade", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.forbes.com/sites/sarwantsingh/2025/07/14/indias-economic-juggernaut-on-way-to-becoming-the-3rd-largest-economy/", "title": "India's Economic Juggernaut On Way To Becoming The ...", "content": "Urbanization and infrastructure development, expected to expand markets, boost productivity, attract investment, and create millions of new jobs, will be th

In [12]:
get_response("Can you search for AI agents and explain them in detail?", thread_id)

================================ Human Message =================================

Can you search for AI agents and explain them in detail?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (vx7vhxrv2)
 Call ID: vx7vhxrv2
  Args:
    query: AI agents
    search_depth: advanced
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "AI agents", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://aws.amazon.com/what-is/ai-agents/", "title": "What are AI Agents? - Artificial Intelligence - AWS", "content": "An artificial intelligence (AI) agent is a software program that can interact with its environment, collect data, and use that data to perform self-directed tasks that meet predetermined goals. Humans set goals, but an AI agent independently chooses the best actions it needs to perform to achieve those goals. For examp

In [14]:
get_response("Generate a code in python to check if prime or not use efficient algorithm", thread_id)

================================ Human Message =================================

Generate a code in python to check if prime or not use efficient algorithm
================================== Ai Message ==================================

import math

def is_prime(n):
    if n <= 1:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    sqrt_n = math.isqrt(n)
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

# Test the function
print(is_prime(25))  # False
print(is_prime(23))  # True
print(is_prime(37))  # True
print(is_prime(48))  # False
